In [10]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from spatialentropy import altieri_entropy, leibovici_entropy
from scipy.stats import entropy
import numpy as np
import gc
import shapely

from IPython.display import clear_output

from tqdm import tqdm
tqdm.pandas()


In [11]:
gemeenten = gpd.read_parquet("data/gemeenten_amenities.parquet")
# wijken = gpd.read_parquet("data/wijken.parquet")

In [12]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

def wk_total_amenities_entropy(gm_name, wijkarea, filter_i):
    L0_BLACKLIST, L1_BLACKLIST = ec.getfilter(filter_i)
    amenity_gdf = gpd.read_parquet(f"data/gm_amenities/amenities_{gm_name}.parquet")
    
    # filter out amenities not in the wijk
    amenity_gdf = amenity_gdf[amenity_gdf.within(wijkarea)]
    amenity_gdf.reset_index(drop=True, inplace=True)
    
    # apply filters
    amenity_gdf = amenity_gdf[~amenity_gdf.L0_category.isin(L0_BLACKLIST)]
    if L1_BLACKLIST:
        for key, value in L1_BLACKLIST.items():
            amenity_gdf = amenity_gdf[
                ~(
                    (amenity_gdf.L0_category == key)
                    & (amenity_gdf.L1_category.isin(value))
                )
            ]
    
    # total number of amenities
    total_amenities = len(amenity_gdf)
    
    points = [[point.x, point.y] for point in amenity_gdf.geometry]
    
    # calculate entropy
    L0 = amenity_gdf.loc[:, f"L0_category"].values
    L1 = amenity_gdf.loc[:, f"L1_category"].values
    # L0_entropy = _get_shannon_entropy(L0, base=2)
    # L1_entropy = _get_shannon_entropy(L1, base=2)
    try:
        L0_entropy = altieri_entropy(points, L0, base=2).entropy
    except:
        L0_entropy = 0
    try:
        L1_entropy = altieri_entropy(points, L1, base=2).entropy
    except:
        L1_entropy = 0
    
    del points, amenity_gdf, L0, L1
    gc.collect()
    
    return total_amenities, L0_entropy, L1_entropy

In [13]:
for filter in [1,2]:
    for part in [2]:
        wijken = gpd.read_parquet(f"data/wijken_parts/wijken_{part}a.parquet")

        for i, wijk in tqdm(wijken.iterrows(), total=len(wijken)):
            gm_name = wijk.gemeentenaam
            wijkarea = wijk.geometry
            total_amenities, L0_entropy, L1_entropy = wk_total_amenities_entropy(gm_name, wijkarea, filter)
            
            # wijken.at[i, f"total_amenities_{filter}"] = total_amenities
            wijken.at[i, f"L0_altieri_{filter}"] = L0_entropy
            wijken.at[i, f"L1_altieri_{filter}"] = L1_entropy
            
            del total_amenities, L0_entropy, L1_entropy
            gc.collect()

    wijken.to_parquet(f"data/wijken_parts/wijken_{part}a.parquet")

  9%|▉         | 50/554 [00:28<03:00,  2.80it/s]/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
 29%|██▉       | 162/554 [01:33<03:06,  2.11it/s]/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/h

In [14]:
wijken.head(3)

,wijkcode,wijknaam,gemeentecode,gemeentenaam,IND_WBI,H2O,OAD,STED,BEV_DICHTH,AANT_INW,...,L1_shannon_1,total_amenities_2,L0_shannon_2,L1_shannon_2,L0_altieri_0,L1_altieri_0,L0_altieri_1,L1_altieri_1,L0_altieri_2,L1_altieri_2
1108,WK040006,Wijk 06 Het Koegras,GM0400,Den Helder,1.0,NEE,583.0,4.0,147.0,1995.0,...,4.150952,53.0,2.778189,3.826147,6.042849,8.365911,5.669136,8.252366,5.578922,7.594151
1109,WK040007,Wijk 07 Duinzoom,GM0400,Den Helder,1.0,NEE,583.0,4.0,41.0,340.0,...,2.841489,58.0,2.135307,2.721359,5.437893,6.686396,4.463903,5.802840,4.346402,5.570021
1110,WK040008,Wijk 08 Julianadorp,GM0400,Den Helder,1.0,NEE,1339.0,3.0,2368.0,12870.0,...,3.605283,94.0,2.759255,3.420244,6.369335,7.856264,5.459933,7.430628,5.386451,7.104064
